In [1]:
import numpy as np
import torch

In [31]:
x = torch.arange(12.0, requires_grad=True)
x.requires_grad_(True)

## 计算第一个函数
y = 2 * torch.dot(x, x)
print(y.requires_grad, y)
## 反向传播
y.backward()
## 传播玩之后，可以看x的每个导数
print(x.grad, x.grad == 4 * x)


## 在默认情况下，pytorch会累积梯度，所以要清空一下梯度
x.grad.zero_()
## 现在来计算另一个函数
y = x.sum()
y.backward()
print(x.grad)

True tensor(1012., grad_fn=<MulBackward0>)
tensor([ 0.,  4.,  8., 12., 16., 20., 24., 28., 32., 36., 40., 44.]) tensor([True, True, True, True, True, True, True, True, True, True, True, True])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


假设y是一个向量的话，在深度学习中，计算不是为了计算微分矩阵，所以会报错

In [40]:
x.grad.zero_()
y = x * x
print(y, '\n')

try:
    y.backward()
except Exception as err:
    print("Error: ", err)
    
y = y.sum()
y.backward()
print(x.grad)

tensor([  0.,   1.,   4.,   9.,  16.,  25.,  36.,  49.,  64.,  81., 100., 121.],
       grad_fn=<MulBackward0>) 

Error:  grad can be implicitly created only for scalar outputs
tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22.])


把某些计算移动到记录的计算图以外

In [46]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x 

## z这个时候系统看u是和x无关的常数
z.sum().backward()
print(x.grad == u)

## y还在计算图里面，如果看y.sum(),还是一个高阶项
x.grad.zero_()
y.sum().backward()
print(x.grad == 2 * x)

tensor([True, True, True, True, True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True, True, True, True, True])


即使在使用了python控制流之后，仍可以计算变量自动求导

In [56]:
def f(a):
    
    for i in range(10):
        a = a * 2.
    
    if a.sum() > 1000:
        return a
    else:
        return a/1000

x.grad.zero_()
y = f(x).sum()
y.backward()
print(x.grad)

tensor([1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024., 1024.,
        1024., 1024.])
